In [130]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import mysql.connector


In [131]:
data = pd.read_csv("C:\\Users\\DELL\\Female_Cyber_Cases_100K_Online_Threats.csv",encoding=(' ISO-8859-1'))
data = pd.read_csv("C:\\Users\\DELL\\Downloads\\INFOSYS COURSE\\cybercrime\\Cyber_Cases_100K_Updated.csv",encoding=(' ISO-8859-1'))

# Check for null values and drop them if necessary
data.dropna(inplace=True)


# Display the first few rows of the dataset
print(data.head())


# Encode the target labels into numerical format
priority_mapping = {'High': 0, 'Moderate': 1, 'Low': 2}
data['Priority Level'] = data['Priority Level'].map(priority_mapping)


# Split features and target
X = data['Case Type']
y = data['Priority Level']





   Case ID       Case Type        Date      Time Priority Level  \
0        1  Malware Attack  2024-01-05  08:45 AM           High   
1        2        Phishing  2017-07-14  02:15 AM       Moderate   
2        3   SQL Injection  2017-10-16  04:40 AM            Low   
3        4   Cryptojacking  2018-09-26  11:56 PM            Low   
4        5  Insider Threat  2019-02-12  06:40 AM       Moderate   

                                         Description      Affected Systems  \
0              Detected ransomware encrypting files.             Endpoints   
1        Phishing incident involving multiple users.      Internal systems   
2   SQL Injection incident involving multiple users.             Endpoints   
3   Cryptojacking incident involving multiple users.  Cloud infrastructure   
4  Insider Threat incident involving multiple users.        Email accounts   

           Location         Reported By               Detection Method  \
0      Delhi, India            Employee               

In [132]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [133]:
# Vectorize the complaint text using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [134]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_vec, y_train)

RandomForestClassifier(random_state=42)

In [135]:

# Make predictions
predictions = model.predict(X_test_vec)
# Print the classification report
print(classification_report(y_test, predictions, target_names=['High', 'Moderate', 'Low']))



              precision    recall  f1-score   support

        High       0.33      0.20      0.25      6755
    Moderate       0.33      0.29      0.31      6627
         Low       0.33      0.50      0.40      6659

    accuracy                           0.33     20041
   macro avg       0.33      0.33      0.32     20041
weighted avg       0.33      0.33      0.32     20041



In [136]:
import pymysql
from sqlalchemy import create_engine

# Define the MySQL connection details
host = 'vultr-prod-cfbaea5c-14b5-4c87-b3e2-9929eed22e05-vultr-prod-4489.vultrdb.com'
user = 'vultradmin'
password = 'special_password'
database = 'defaultdb'
port = 16751

# Use PyMySQL with SQLAlchemy
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')

# Test the connection
try:
    with engine.connect() as connection:
        result = connection.execute("SELECT 1")
        print("Connection successful, test query result:", result.fetchone())
except Exception as e:
    print("Error connecting to the database:", e)

index = 0


Connection successful, test query result: (1,)


In [137]:
import mysql.connector
# Connect to MySQL database

db = mysql.connector.connect(
    host="vultr-prod-cfbaea5c-14b5-4c87-b3e2-9929eed22e05-vultr-prod-4489.vultrdb.com",       # Hostname of the database server
    user="vultradmin",   # Your MySQL username
    password="special_password", # Your MySQL password
    database="defaultdb", # Name of the database you want to connect to
    port=16751,         # Port number (usually 3306 for MySQL)
    auth_plugin='mysql_native_password'  # Authentication plugin if necessary
)

if db.is_connected():
    print("Connection successful")
else:
    print("Connection failed")
cursor = db.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS model_complaints (
    model_complaint_id INT PRIMARY KEY AUTO_INCREMENT,
    complaint_id INT UNSIGNED NOT NULL,
    user_id INT UNSIGNED NOT NULL,
    description TEXT NOT NULL,
    `low_priority` INT DEFAULT 0,    -- INT to hold priority levels
    `medium_priority` INT DEFAULT 0,
    `high_priority` INT DEFAULT 0,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (complaint_id) REFERENCES complaints(id),
    FOREIGN KEY (user_id) REFERENCES users(id)
);
''')
create_gov_send_table_query = '''
    CREATE TABLE IF NOT EXISTS gov_send (
        id INT PRIMARY KEY AUTO_INCREMENT,
        description TEXT NOT NULL,
        user_id INT UNSIGNED NOT NULL,
        complaint_id INT UNSIGNED NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES users(id),
        FOREIGN KEY (complaint_id) REFERENCES complaints(id)
    );
    '''

cursor.execute(create_gov_send_table_query)

db.commit()
index = 0

Connection successful


In [138]:

query = "SELECT * FROM complaints"
userComplaint = pd.read_sql(query, db)
X_test = userComplaint['description'] 
user_ids = userComplaint['user_id'] 
complaint_ids = userComplaint['id'] 
# print(X_test)
user_ids = user_ids.astype(int)
complaint_ids = complaint_ids.astype(int)


for i, user_complaint in enumerate(X_test):
    # Assuming predictions is a list or array of predictions corresponding to each complaint
    high_priority = 1 if predictions[i] == 0 else 0
    medium_priority = 1 if predictions[i] == 1 else 0
    low_priority = 1 if predictions[i] == 2 else 0
    # Convert all variables to standard Python types
    cursor.execute('''
    INSERT INTO model_complaints (description, `high_priority`, `medium_priority`, `low_priority`, user_id, complaint_id) 
    VALUES (%s, %s, %s, %s, %s, %s)
    ''', (X_test[i], int(high_priority), int(medium_priority), int(low_priority), int(user_ids.iloc[i]), int(complaint_ids.iloc[i])))
    if high_priority == 1:
        cursor.execute('''
        INSERT INTO gov_send (description, user_id, complaint_id)
        VALUES (%s, %s, %s)
        ''', (X_test[i], int(user_ids.iloc[i]), int(complaint_ids.iloc[i])))

db.commit()

C:\Users\DELL\AppData\Local\Temp\ipykernel_3448\3280068286.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  userComplaint = pd.read_sql(query, db)


In [142]:

data = "SELECT * FROM gov_send"
tableData = pd.read_sql(data, db)
print(tableData)

    id                                        description  user_id  \
0    1                 Unable to access my email account.        1   
1    2   Received wrong item; urgent need for resolution.        1   
2    3  Unauthorized transactions detected on my account.        1   
3    4                     Leaking faucet in the kitchen.        2   
4    5  Loud music coming from the neighbor's apartmen...        1   
5    6              Graffiti on the side of the building.        1   
6    7         Overflowing trash bins in the common area.        3   
7    8                     Leaking faucet in the kitchen.        2   
8    9  Unauthorized vehicle parked in my designated s...        3   
9   10                Neighbor's dog barking excessively.        2   
10  11  Unauthorized vehicle parked in my designated s...        3   
11  12              Graffiti on the side of the building.        1   
12  13   Suspicious activity reported in the parking lot.        4   
13  14  Unauthorized

C:\Users\DELL\AppData\Local\Temp\ipykernel_3448\3424523756.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tableData = pd.read_sql(data, db)


In [ ]:

# tables = "SHOW TABLES"
# tab = pd.read_sql(tables, db)
# print(tab)

  Tables_in_defaultdb
0          complaints
1            gov_send
2    model_complaints
3               users


C:\Users\DELL\AppData\Local\Temp\ipykernel_3448\3314326901.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tab = pd.read_sql(tables, db)


In [141]:

# drop_tab = "DROP TABLE gov_send"
# drop_tables = pd.read_sql(drop_tab, db)
# print(drop_tables)